<a href="https://colab.research.google.com/github/screid/Pagina-kcpm/blob/main/Codigo_para_obtener_georeferencias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopy

In [2]:
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [3]:
data = pd.read_csv('patentes_ejemplo.csv')

In [4]:
# No corras esto. Esto lo hice solo para agregar las columnas adicionales de pais, región y comuna.
data['COMUNA'] = 'Providencia'
data['REGION'] = 'Metropolitana'
data['PAIS'] = 'Chile'

In [5]:
data.head()

,FECHA OTORGAMIENTO,ROL,NOMBRE,DIRECCION,GIRO O ACTIVIDAD,COMUNA,REGION,PAIS
0,30/04/2025,2-201409,FUNERARIA IVÁN MARTÍNEZ IV SPA,SANTA MARIA 0180,OFICINA ADMINISTRATIVA DE SERVICIOS FUNERARIOS...,Providencia,Metropolitana,Chile
1,30/04/2025,2-201404,STUDIO GREECE SPA,AVENIDA NUEVA PROVIDENCIA 2606 LOCAL 4,"SERVICIOS DE BARBERÍA, PELUQUERÍA, MANICURA, V...",Providencia,Metropolitana,Chile
2,30/04/2025,2-201405,APT COMERCIALIZADORA SPA,ANTONIO BELLET 193 OFICINA 1210,OFICINA TRIBUTARIA DE VENTA AL POR MENOR DE PR...,Providencia,Metropolitana,Chile
3,30/04/2025,2-201406,VARQUEZ SPA,ANTONIO BELLET 193 OFICINA 1210,OFICINA TRIBUTARIA DE VENTA AL POR MENOR DE VE...,Providencia,Metropolitana,Chile
4,30/04/2025,2-201402,"SERVICIOS, CONSULTORIAS E INVERSIONES DEBEELEN...",DR. MANUEL BARROS BORGOÑO 110 OFICINA 805,"SERVICIOS DE ASESORÍAS, TERÁPIAS DE SALUD MENT...",Providencia,Metropolitana,Chile


In [6]:
# Función para limpiar la dirección
def limpiar_direccion(direccion):
    # Convierte a mayúsculas por consistencia
    direccion = direccion.upper()
    # Elimina palabras como LOCAL, OFICINA, DEPTO, etc., seguidas de números u otros caracteres.
    # Si necesitas eliminar otras, colocalas siguiendo el patrón COSO1|COSO2. Este paso es importante, o la API no lo encuentra.
    direccion = re.sub(r'\b(LOCAL|OFICINA|DEPTO|DPTO|DEPARTAMENTO)\b[\s\S]*', '', direccion)
    # Elimina espacios finales
    return direccion.strip()

In [7]:
# Construir dirección completa para geocodificar
def construir_direccion_completa(row):
    base = limpiar_direccion(row['DIRECCION'])
    comuna = row['COMUNA']
    region = row['REGION']
    pais = row['PAIS']
    return f"{base}, {comuna}, {region}, {pais}"

In [8]:
# Aplicar transformación
data['DIRECCION_GEO'] = data.apply(construir_direccion_completa, axis=1)

In [9]:
# Configurar geocoder
geolocator = Nominatim(user_agent="geoapi_usach", timeout=10)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=3, error_wait_seconds=2.0)

In [10]:
# Función de geocodificación robusta
def obtener_coordenadas(direccion):
    try:
        ubicacion = geocode(direccion)
        if ubicacion:
            return pd.Series([ubicacion.latitude, ubicacion.longitude])
        else:
            return pd.Series([None, None])
    except Exception as e:
        print(f"Error con dirección '{direccion}': {e}")
        return pd.Series([None, None])

In [11]:
# Aplicar
data[['latitud', 'longitud']] = data['DIRECCION_GEO'].apply(obtener_coordenadas)

In [12]:
data.head()

,FECHA OTORGAMIENTO,ROL,NOMBRE,DIRECCION,GIRO O ACTIVIDAD,COMUNA,REGION,PAIS,DIRECCION_GEO,latitud,longitud
0,30/04/2025,2-201409,FUNERARIA IVÁN MARTÍNEZ IV SPA,SANTA MARIA 0180,OFICINA ADMINISTRATIVA DE SERVICIOS FUNERARIOS...,Providencia,Metropolitana,Chile,"SANTA MARIA 0180, Providencia, Metropolitana, ...",-33.420450,-70.614631
1,30/04/2025,2-201404,STUDIO GREECE SPA,AVENIDA NUEVA PROVIDENCIA 2606 LOCAL 4,"SERVICIOS DE BARBERÍA, PELUQUERÍA, MANICURA, V...",Providencia,Metropolitana,Chile,"AVENIDA NUEVA PROVIDENCIA 2606, Providencia, M...",-33.427879,-70.617317
2,30/04/2025,2-201405,APT COMERCIALIZADORA SPA,ANTONIO BELLET 193 OFICINA 1210,OFICINA TRIBUTARIA DE VENTA AL POR MENOR DE PR...,Providencia,Metropolitana,Chile,"ANTONIO BELLET 193, Providencia, Metropolitana...",-33.425374,-70.617923
3,30/04/2025,2-201406,VARQUEZ SPA,ANTONIO BELLET 193 OFICINA 1210,OFICINA TRIBUTARIA DE VENTA AL POR MENOR DE VE...,Providencia,Metropolitana,Chile,"ANTONIO BELLET 193, Providencia, Metropolitana...",-33.425374,-70.617923
4,30/04/2025,2-201402,"SERVICIOS, CONSULTORIAS E INVERSIONES DEBEELEN...",DR. MANUEL BARROS BORGOÑO 110 OFICINA 805,"SERVICIOS DE ASESORÍAS, TERÁPIAS DE SALUD MENT...",Providencia,Metropolitana,Chile,"DR. MANUEL BARROS BORGOÑO 110, Providencia, Me...",-33.429666,-70.618663
